In [0]:
import pyspark.pandas as ps
df = ps.read_csv("/databricks-datasets/COVID/covid-19-data")
df["date"] = ps.to_datetime(df['date'],errors='coerce')
df["cases"] = df["cases"].astype(int)
display(df)

In [0]:
df.count()

In [0]:
df.columns

In [0]:
df.describe()

In [0]:
from databricks import automl
import logging

In [0]:
logging.getLogger("py4j").setLevel(logging.WARNING)

In [0]:
summary = automl.forecast(df,target_col='cases',time_col='date', horizon=30, frequency="d", primary_metric='mdape', output_database='default', timeout_minutes=15)

In [0]:
print(summary)

In [0]:
print(summary.output_table_name)

In [0]:
forecast_pd = spark.table(summary.output_table_name)
display(forecast_pd)

In [0]:
import mlflow.pyfunc
from mlflow.tracking import MlflowClient
run_id = MlflowClient()
trial_id = summary.best_trial.mlflow_run_id

model_uri = "runs:/{run_id}/model".format(run_id=trial_id)
pyfunc_model=mlflow.pyfunc.load_model(model_uri)

In [0]:
forecasts = pyfunc_model._model_impl.python_model.predict_timeseries()
display(forecasts)